In [7]:
import tweepy
import nltk
import pandas as pd
import pickle
import numpy as np 
import re 
import os
import random
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from statistics import mode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [8]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [9]:
# Open dataset 
dataset = pd.read_csv('../resource/labeled_data.csv')
# class: 0 - hate speech 1 - offensive language 2 - neither
dt_transformed = dataset[['class', 'tweet']]

In [10]:
# df_train, df_test = train_test_split(dt_transformed, test_size = 0.15, random_state = 42, stratify=dt_transformed['class'])
# print(f"""
# Training table have {df_train.shape[0]} rows and {df_train.shape[1]} column 
# Test table have {df_test.shape[0]} rows and {df_test.shape[1]} column
# """)
# df_train['class'].value_counts().plot(kind='barh',color='c');df_test['class'].value_counts().plot(kind='barh',color='m')
dt_transformed['class'].values[0]

2

In [11]:
all_words = [] 
documents = [] 

def processing_text_to_pos_tag(row_table):
    all_stopwords = stopwords.words('english')
    t_data = re.sub("^RT.", "", row_table[1])  # removing RTT
    t_data = re.sub("@[0-9A-za-z:]+", "", t_data)  # removing @user
    t_data = re.sub("https?://[A-Za-z0-9./&#]+", "", t_data)  # removing http links
    t_data = re.sub("&#[0-9;]+", " ", t_data) # removing emojis
    t_data = t_data.lower()
    t_data = t_data.split()
    ps = PorterStemmer()
    t_data = [ps.stem(word) for word in t_data if not word in set(all_stopwords) if len(word) > 2]
    t_data = ' '.join(t_data)
    row_table[0] = int(row_table[0])
    if row_table[0] == 0:
        row_table[0] = 1
    return row_table[0], t_data

In [12]:
# allowed_word_types = ["J"]

for row in dt_transformed.values:
    c,t = processing_text_to_pos_tag(row)
    documents.append( (t,c) )
    words = word_tokenize(t)
#     pos_t = nltk.pos_tag(words)
    for w in words:
        all_words.append(w)
        
save_documents = open("../pickled_algos/documents.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()

all_words = nltk.FreqDist(all_words)
word_features = list(all_words.keys())[:5000]

save_word_features = open("../pickled_algos/word_features5k.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()

In [13]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [14]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]

In [15]:
# len(featuresets)
testing_set = featuresets[11000:]
training_set = featuresets[:11000]

In [16]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
# classifier.show_most_informative_features(15)

save_classifier = open("../pickled_algos/originalnaivebayes5k.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

save_classifier = open("../pickled_algos/MNB_classifier5k.pickle","wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

save_classifier = open("../pickled_algos/BernoulliNB_classifier5k.pickle","wb")
pickle.dump(BernoulliNB_classifier, save_classifier)
save_classifier.close()

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

save_classifier = open("../pickled_algos/LogisticRegression_classifier5k.pickle","wb")
pickle.dump(LogisticRegression_classifier, save_classifier)
save_classifier.close()

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

save_classifier = open("../pickled_algos/LinearSVC_classifier5k.pickle","wb")
pickle.dump(LinearSVC_classifier, save_classifier)
save_classifier.close()

SGDC_classifier = SklearnClassifier(SGDClassifier())
SGDC_classifier.train(training_set)
print("SGDClassifier accuracy percent:",nltk.classify.accuracy(SGDC_classifier, testing_set)*100)

save_classifier = open("../pickled_algos/SGDC_classifier5k.pickle","wb")
pickle.dump(SGDC_classifier, save_classifier)
save_classifier.close()

Original Naive Bayes Algo accuracy percent: 93.0494086918668
MNB_classifier accuracy percent: 92.96234491765219
BernoulliNB_classifier accuracy percent: 91.82326053834434
LogisticRegression_classifier accuracy percent: 95.5017049989117
LinearSVC_classifier accuracy percent: 94.48596096640789
SGDClassifier accuracy percent: 95.32757745048248


In [17]:
voted_classifier = VoteClassifier(
                                  classifier,
                                  LinearSVC_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)

In [36]:
text_ = ' '
def out(text):
    feats = find_features(text)
    return voted_classifier.classify(feats),voted_classifier.confidence(feats)

# c,cc = out(text_)
# if cc <= 0.8:
#     print('Sem certeza')
# else:
#     if c == 1:
#         print('Discurso de odio ou ofensiva')
#     else:
#         print('Normal')